<a href="https://colab.research.google.com/github/santhoshmohan8/MachineLearning/blob/main/Logistic_Spam_Mail_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read mail data and check if any NaN and replace any missing values

In [120]:
import pandas as pd
import numpy as np
from google.colab import data_table
from vega_datasets import data
data_table.enable_dataframe_formatter()

spam_data = pd.read_csv("mail_data.csv")
spam_data.head()

spam_data.isna().sum()
print(spam_data.shape,"\n\n\n")
spam_data.info()
print(spam_data.describe(),"\n\n")
spam_data = spam_data.where((pd.notnull(spam_data)),'') # replace NaN values if any
print("Event rate for target : \n",spam_data.Category.value_counts()/spam_data.shape[0]*100)

(5572, 2) 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
       Category                 Message
count      5572                    5572
unique        2                    5157
top         ham  Sorry, I'll call later
freq       4825                      30 


Event rate for target : 
 ham     86.593683
spam    13.406317
Name: Category, dtype: float64


Target variable is categorical, so converting them using Label Encoder

*inverse_transform - Checking what is the encoded value for each categories*

In [98]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
target_encoded = encoder.fit_transform(spam_data['Category'])

spam_data = pd.concat([spam_data,pd.Series(target_encoded)], axis=1).drop(columns=['Category']).rename(columns={0:'Category'})
spam_data.head()

target_decoded = encoder.inverse_transform(target_encoded)
print(pd.DataFrame(np.vstack((target_decoded,target_encoded)).T).groupby([0,1]).nunique())

Empty DataFrame
Columns: []
Index: [(ham, 0), (spam, 1)]


Train test split & TFIDF vectorizer for text features

In [99]:
from sklearn.model_selection import train_test_split
x = spam_data['Message']
y = spam_data['Category']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=3)

from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(stop_words='english',min_df = 1,lowercase = True)

x_train_feature = vector.fit_transform(x_train)
x_train_feature.shape
x_test_feature = vector.transform(x_test)
x_test_feature.shape

y_train = y_train.astype('int')
y_test = y_test.astype('int')

Model performance metrics - Function

In [124]:
from sklearn import metrics
def model_performance(y_test,y_test_predict):
  print("\n\nConfusion matrix :\n",metrics.confusion_matrix(y_test,y_test_predict))
  print("\n\nClassification report :\n",metrics.classification_report(y_test,y_test_predict))
  print("\nAccuracy score :",metrics.accuracy_score(y_test,y_test_predict))
  print("MAPE :",1-metrics.accuracy_score(y_test,y_test_predict))
  print("Hamming loss :",metrics.hamming_loss(y_test,y_test_predict)) # fraction of labels incorrectly predicted
  print("Jaccard score :",metrics.jaccard_score(y_test,y_test_predict)) # Jaccard score - not a good metric for imbalanced dataset
  print("Log loss :",metrics.log_loss(y_test,y_test_predict)) # log loss or cross entropy loss

Baseline Predictions

In [105]:
from sklearn.dummy import DummyClassifier
model = DummyClassifier(strategy='uniform').fit(x_train_feature,y_train)
y_train_predict = model.predict(x_train_feature)
y_test_predict = model.predict(x_test_feature)
model_performance(y_test,y_test_predict)

Confusion matrix :
 [[489 471]
 [ 81  74]]


Classification report :
               precision    recall  f1-score   support

           0       0.86      0.51      0.64       960
           1       0.14      0.48      0.21       155

    accuracy                           0.50      1115
   macro avg       0.50      0.49      0.43      1115
weighted avg       0.76      0.50      0.58      1115


Accuracy score : 0.5049327354260089
MAPE : 0.4950672645739911
Hamming loss : 0.49506726457399103
Jaccard score : 0.1182108626198083
Log loss : 17.84403288860329


Logistic model with Hyperparameter tuning

In [150]:
from sklearn.linear_model import LogisticRegression
model_list = [
    LogisticRegression(),
    LogisticRegression(C=1, solver='newton-cg', penalty='l2', max_iter=100),
    LogisticRegression(C=1, solver='lbfgs', penalty='l2', max_iter=100),
    LogisticRegression(C=1, solver='sag', penalty='l2', max_iter=100),
    LogisticRegression(C=1, solver='saga', penalty='l2', max_iter=100)
]

for model in model_list:
  model.fit(x_train_feature,y_train)
  y_train_predict = model.predict(x_train_feature)
  y_test_predict = model.predict(x_test_feature)
  model_performance(y_test,y_test_predict)



Confusion matrix :
 [[960   0]
 [ 38 117]]


Classification report :
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       960
           1       1.00      0.75      0.86       155

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115


Accuracy score : 0.9659192825112107
MAPE : 0.03408071748878927
Hamming loss : 0.034080717488789235
Jaccard score : 0.7548387096774194
Log loss : 1.2283935684183427


Confusion matrix :
 [[960   0]
 [ 38 117]]


Classification report :
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       960
           1       1.00      0.75      0.86       155

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115


Accuracy score : 0.9659192825112107


Parameter tuning with Grid Search CV

In [156]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

from sklearn.model_selection import GridSearchCV
model_cv = {'C':[0.1,1,10],
              'solver':['newton-cg','lbfgs','sag','saga'],
              'penalty':['l2'],
              'max_iter':[100]}

model_GSCV = GridSearchCV(estimator=model,param_grid = model_cv,cv=3).fit(x_train_feature,y_train)
print(model_GSCV.best_params_)
print(model_GSCV.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
0.9697105645961636


Using best parameters to build a model

In [154]:
model = LogisticRegression(**model_GSCV.best_params_).fit(x_train_feature,y_train)
y_train_predict = model.predict(x_train_feature)
y_test_predict = model.predict(x_test_feature)
model_performance(y_test,y_test_predict)



Confusion matrix :
 [[959   1]
 [ 19 136]]


Classification report :
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       960
           1       0.99      0.88      0.93       155

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115


Accuracy score : 0.9820627802690582
MAPE : 0.017937219730941756
Hamming loss : 0.017937219730941704
Jaccard score : 0.8717948717948718
Log loss : 0.6465229307464961
